In [1]:
# Importing used packages and model
from cobra.io import read_sbml_model
from cobra import Reaction, Metabolite
model = read_sbml_model('iMT1026-v4.xml')
%run -i create_pathway.ipynb  # runs other script with included function to alter model

In [2]:
with model:
    model2 = model.copy()
    create_pathway(model2,1)


Added beta alanine pathway


***Methanol assimilation***:

As mentioned, utilising methanol as a substrate could be a great alternative and a step towards more for sustainable processes. However, both growth and product titers are significantly lowered compared to other substrates. Miao manages to produce 5.6 g/L of beta-alanine in P. pastoris grown on methanol which they state is the highest metabolite titer reached for the conditions. Up to 43.12 has been reached with glucose.

For investigation of the drawbacks behind methanol utilisation the pathway is drawn in escher for fluxes under optimized growth. 

In [3]:
# Realistic methanol uptake rates cf.Tomàs‐Gamisans 2018
medium = model2.medium
medium['Ex_meoh'] = 6
medium['Ex_glyc'] = 0
model2.medium = medium
model2.objective = model2.reactions.Ex_biomass
solution = model2.optimize()

In [4]:
import escher
from IPython.display import display
b = escher.Builder(map_json = 'iMM904.Central_carbon_metabolism.json', reaction_data = solution.fluxes.to_dict(),highlight_missing=True, model=model2)
display(b)

Builder(highlight_missing=True, reaction_data={'ADPtn': 0.0, 'ATPtn': 0.0, 'FACOAE140': -3.245994347871679e-20…

As seen, approximately a third of the formaldehyde flux is taken down the path and oxidized in reactions FALDH2 and FDH to produce reducing power in form of NADH (82.2% NADH of all contribution). Shown in Schroer 2009 that FLD is the main bottleneck for NADH generation. At this point it becomes clear why  why malonate is an interesting product of choice through the use of the beta-alanine pathway; the end step is a NADH regenerator. Constraining the biomass formation to 50% and optimizing for malonate production, 29.18% of the necessary NADH is generated by MSADH and only 30.3% is thus needed from the formaldehyde oxidation steps. With the same setup, but instead having beta-alanine as the product optimized for, FALDH2 and FDH have to constitute 67.58% of NADH generation (maybe put a table). The resulting yield of beta-alanine becomes 0.36 g ala_B/g meoh while it almost doubles for malonate (0.62 g mln/g meoh).

It is clear that the NADH/NADPH balance is important for optimizing growth and improving productivity. Much experimentation with manual cofactor swaps was peformed to check for more optimal routes. It was managed to improve the base growth rate by swapping NADH with NADPH in GAPD in glycolysis, but only by 0.0022 h^-1 and it did not improve the phenotypic phase plane significantly. OPTSwap was employed for algorithmic predictions instead but with no result either (show in github).



***Optimizing the use of malonic-semialdehyde***

OPTGene and OPTknock was used to discover possible knockout strategies. As with OPTSwap, no targets were identified (show in github). However, in an in-vivo context there are multiple considerations to be made. First off, FVA reveals that malonic-semialdehyde is not only used for malonate production but also conversion into acetyl-coa through MMSAD3. The gene coding for this reaction is should be knocked out to avoid this occuring in the real biological system. Computationally, doing this does not affect the phenotypic phase plane either

In a study (Borodina 2014) about 3-hydroxypropanoate production in S. cerevisiae, screening was peformed to identify the best hetereologous reaction for beta alinine to malonic-semialdehyde conversion (BABAT). Even though the reaction APAT2r is present in both P. pastoris and S. cerevisiae the conclusion was that a gene from B cerus should be inserted. However, FVA revealed that APAT2r and BABAT were equally effective. This goes to show the limits of non-constrained Fva. Here it points to APAT2r being sufficient for in vivo msa production, contrary to the findings of Borodina.

***Improving the aspartate supply***

Aspartate is cf Miao 2021 reported to have a large metabolic pool size in P pastoris also makes the reaction favorable. It is the goal to further improve this property to ensure a higher flux towards beta-alanine. Song 2016 engineered E.coli for 3HP and MA production. Part of the strategy was to facilitate aspartate formation by introducing direct amination of fumarate by aspartase. This was also implemented in the model through the reaction ASPAML but yet again with no significant results other than being a possible route in Fva.

Miao 2021 also works with improving aspartate availability but instead by improving supply from OAA. From the escher plots and fva it should be noticed that the opposite is occuring. By strengthening the OAA production through CO2 fixation these fluxes could possibly be redirected. This was done by introducing phosphoenolpyruvate carboxykinase from A. succinogenes (Hu 2018). How much it will improve the productivity in a real organism is however interesting, since the thermodynamics of the reaction may not allow such a big flux through this reaction.